<a href="https://colab.research.google.com/github/ander-yamamoto/dio-bairesdev/blob/main/assistente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Microphone code:
https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb#scrollTo=XZlWKbI4A1Rp

In [13]:
!pip -q install git+https://github.com/openai/whisper.git librosa ffmpeg-python gTTS wikipedia pyjokes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.0 MB/s eta 0:00:00


In [4]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
import io, ffmpeg, librosa, whisper
import numpy as np
import soundfile as sf

AUDIO_HTML = """
<script>
let btn;
let recorder;
let chunks = [];
let stream;

async function initRecorder() {
  stream = await navigator.mediaDevices.getUserMedia({audio:true});
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = e => { if (e.data.size > 0) chunks.push(e.data); };
}

async function recordOnce() {
  return new Promise(async (resolve, reject) => {
    try {
      if (!btn) {
        btn = document.createElement('button');
        btn.textContent = 'Gravar';
        btn.style = 'padding:8px 12px; font-size:14px;';
        document.body.appendChild(btn);
      }

      btn.onclick = async () => {
        if (!recorder || recorder.state === "inactive") {
          // iniciar gravação
          chunks = [];
          await initRecorder();
          recorder.start();
          btn.textContent = "Stop";
        } else if (recorder.state === "recording") {
          // parar gravação
          recorder.onstop = async () => {
            const blob = new Blob(chunks, {type:'audio/webm'});
            const reader = new FileReader();
            reader.onloadend = () => {
              btn.textContent = "Gravar";
              resolve(reader.result);
            };
            reader.readAsDataURL(blob);
            stream.getAudioTracks().forEach(t => t.stop());
          };
          recorder.stop();
        }
      };
    } catch (err) { reject(err); }
  });
}
</script>
"""

def get_audio_blob():
    display(HTML(AUDIO_HTML))
    data = eval_js("recordOnce()")  # aguarda clique -> gravação -> stop
    binary = b64decode(data.split(',')[1])

    # webm/opus -> wav 16k mono
    out, _ = (
        ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav', ac=1, ar='16000')
        .run(input=binary, capture_stdout=True, capture_stderr=True)
    )
    return out

# ==== loop de teste ====
model = whisper.load_model("base")


def get_audio():
    print("👉 Clique em 'Gravar', depois 'Stop'.")
    wav_bytes = get_audio_blob()

    audio_np, sr = sf.read(io.BytesIO(wav_bytes), dtype='float32')
    if audio_np.ndim == 2:
        audio_np = np.mean(audio_np, axis=1)
    if sr != 16000:
        audio_np = librosa.resample(audio_np, orig_sr=sr, target_sr=16000)
        sr = 16000

    result = model.transcribe(audio_np, fp16=False, language="en")
    #print("📝 Transcrição:", result["text"].strip())
    #print("-"*60)

    return result["text"].strip()


In [5]:
from IPython.display import Audio, display
from gtts import gTTS
import os

def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = "voice.mp3"
    tts.save(filename)
    display(Audio(filename, autoplay=True))

In [6]:
import wikipedia
import pyjokes
from datetime import datetime

#function to respond to commands
def respond(text):
    print(text)
    if 'youtube' in text:
        speak("What do you want to search for?")
        keyword = get_audio()
        if keyword!= '':
            url = f"https://www.youtube.com/results?search_query={keyword}"
            speak(f"Here is what I have found for {keyword} on youtube")
            print(f"Here is what I have found for {keyword} on youtube")
    elif 'search' in text:
        speak("What do you want to search for?")
        query = get_audio()
        if query !='':
            result = wikipedia.summary(query, sentences=3)
            speak("According to wikipedia")
            print(result)
            speak(result)
    elif 'joke' in text:
        speak(pyjokes.get_joke())
    elif 'what time is it' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'exit' in text:
        speak("Goodbye, till next time")
        exit()

In [20]:
respond('what time is it')

what time is it
19:45 PM


In [9]:
while True:
    text = get_audio().lower()
    if 'exit' in text:
        speak("Goodbye, till next time")
        break
    respond(text)

👉 Clique em 'Gravar', depois 'Stop'.


search.


👉 Clique em 'Gravar', depois 'Stop'.


Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who is the 47th president of the United States. A member of the Republican Party, he served as the 45th president from 2017 to 2021.
Born into a wealthy family in New York City, Trump graduated from the University of Pennsylvania in 1968 with a bachelor's degree in economics.


👉 Clique em 'Gravar', depois 'Stop'.
